In [ ]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the dataset without the "Output Label" column
csv_file_path = 'syllogisms_temp.csv'
data = pd.read_csv(csv_file_path)

# Preparing the GPT-2 model and tokenizer
model_name = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Defining a function to generate predictions for each row
def generate_prediction(row):
    # Construct a 2-shot prompt with explicit examples
    five_shot_prompt = (
        "Premise 1: All birds lay eggs.\n"
        "Premise 2: Penguins are birds.\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion: Penguins lay eggs.\n\n"
        "Premise 1: All animals that swim are aquatic.\n"
        "Premise 2: All fishes are animals.\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion: All fishes are aquatic.\n\n"
        "Premise 1: All flowers produce pollen.\n"
        "Premise 2: Sunflowers are flowers.\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion: Sunflowers produce pollen.\n\n"
        "Premise 1: All dogs are mammals.\n"
        "Premise 2: Some poodles are dogs.\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion: Some poodles are mammals.\n\n"
        "Premise 1: All encryption algorithms that use asymmetric keys require a public and a private key.\n"
        "Premise 2: All RSAs are encryption algorithms that use asymmetric keys.\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion: All RSAs require a public and a private key.\n\n"
    )

    # Construct the input prompt for the specific syllogism
    prompt = (
        f"{five_shot_prompt}"
        f"Premise 1: {row['Premise 1']}\n"
        f"Premise 2: {row['Premise 2']}\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion:"
    )

    # Tokenizing the input
    input_ids = tokenizer.encode(prompt, return_tensors='pt', truncation=True, max_length=1024).to(device)

    # Generating the output with a limit on the number of tokens
    output_ids = model.generate(input_ids, max_length=300)

    # Decoding the output
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    print(output_text)
    # Post-processing to interpret the response consistently
    if "true" in output_text.lower():
        return "TRUE", output_text
    elif "false" in output_text.lower():
        return "FALSE", output_text
    else:
        # Handling unexpected responses gracefully
        return "UNDEFINED", output_text

# Applying this function to each row and collecting results
results = data.apply(generate_prediction, axis=1)
data['Prediction'], data['GPT2 Output'] = zip(*results)

# Saving the revised dataset with both predictions and raw responses
output_csv_path = 'syllogisms_gpt2_2shot.csv'
data.to_csv(output_csv_path, index=False)

print("Results saved to", output_csv_path)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Premise 1: All birds lay eggs.
Premise 2: Penguins are birds.
What is the conclusion drawn from the given two premises?
Conclusion: Penguins lay eggs.

Premise 1: All animals that swim are aquatic.
Premise 2: All fishes are animals.
What is the conclusion drawn from the given two premises?
Conclusion: All fishes are aquatic.

Premise 1: All flowers produce pollen.
Premise 2: Sunflowers are flowers.
What is the conclusion drawn from the given two premises?
Conclusion: Sunflowers produce pollen.

Premise 1: All dogs are mammals.
Premise 2: Some poodles are dogs.
What is the conclusion drawn from the given two premises?
Conclusion: Some poodles are mammals.

Premise 1: All encryption algorithms that use asymmetric keys require a public and a private key.
Premise 2: All RSAs are encryption algorithms that use asymmetric keys.
What is the conclusion drawn from the given two premises?
Conclusion: All RSAs require a public and a private key.

Premise 1: All smartphones are devices that can ac

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Premise 1: All birds lay eggs.
Premise 2: Penguins are birds.
What is the conclusion drawn from the given two premises?
Conclusion: Penguins lay eggs.

Premise 1: All animals that swim are aquatic.
Premise 2: All fishes are animals.
What is the conclusion drawn from the given two premises?
Conclusion: All fishes are aquatic.

Premise 1: All flowers produce pollen.
Premise 2: Sunflowers are flowers.
What is the conclusion drawn from the given two premises?
Conclusion: Sunflowers produce pollen.

Premise 1: All dogs are mammals.
Premise 2: Some poodles are dogs.
What is the conclusion drawn from the given two premises?
Conclusion: Some poodles are mammals.

Premise 1: All encryption algorithms that use asymmetric keys require a public and a private key.
Premise 2: All RSAs are encryption algorithms that use asymmetric keys.
What is the conclusion drawn from the given two premises?
Conclusion: All RSAs require a public and a private key.

Premise 1: All video games require electricity to 

In [ ]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import re

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the dataset without the "Output Label" column
csv_file_path = 'syllogisms_temp.csv'
data = pd.read_csv(csv_file_path)

# Preparing the GPT-2 model and tokenizer
model_name = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Defining a function to generate predictions for each row
def generate_prediction(row):
    # Construct a 5-shot prompt with explicit examples
    five_shot_prompt = (
        "Premise 1: All birds lay eggs.\n"
        "Premise 2: Penguins are birds.\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion: Penguins lay eggs.\n\n"
        "Premise 1: All animals that swim are aquatic.\n"
        "Premise 2: All fishes are animals.\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion: All fishes are aquatic.\n\n"
        "Premise 1: All flowers produce pollen.\n"
        "Premise 2: Sunflowers are flowers.\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion: Sunflowers produce pollen.\n\n"
        "Premise 1: All dogs are mammals.\n"
        "Premise 2: Some poodles are dogs.\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion: Some poodles are mammals.\n\n"
        "Premise 1: All encryption algorithms that use asymmetric keys require a public and a private key.\n"
        "Premise 2: All RSAs are encryption algorithms that use asymmetric keys.\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion: All RSAs require a public and a private key.\n\n"
    )

    # Construct the input prompt for the specific syllogism
    prompt = (
        f"{five_shot_prompt}"
        f"Premise 1: {row['Premise 1']}\n"
        f"Premise 2: {row['Premise 2']}\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion:"
    )

    # Tokenizing the input
    input_ids = tokenizer.encode(prompt, return_tensors='pt', truncation=True, max_length=1024).to(device)

    # Generating the output with a limit on the number of tokens
    output_ids = model.generate(input_ids, max_length=300)

    # Decoding the output
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Finding all conclusion lines
    conclusions = re.findall(r"Conclusion:.*?\.", output_text)

    # Extracting the 6th occurrence
    if len(conclusions) >= 6:
        sixth_conclusion = conclusions[5][11:] # Removing "Conclusion: " (11 characters)
    else:
        sixth_conclusion = "6th conclusion not found."
    print(sixth_conclusion)
    return sixth_conclusion

# Applying this function to each row and collecting results
data['6th Conclusion'] = data.apply(generate_prediction, axis=1)

# Truncate spaces and compare
def compare_conclusions(row):
    conclusion_1 = row['Conclusion'].strip()
    conclusion_6th = row['6th Conclusion'].strip()
    return conclusion_1 == conclusion_6th

# Calculate the final score
score = data.apply(compare_conclusions, axis=1).sum()

# Display the final score
print("Final score:", score)

# Saving the revised dataset with only the 6th conclusion
output_csv_path = 'syllogisms_gpt2_6th_conclusion.csv'
data.to_csv(output_csv_path, index=False)

print("Results saved to", output_csv_path)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 All iPhones require a public and a private key.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 All consoles require electricity to be played.
 All assembly line robots can perform automated tasks.
Final score: 1
Results saved to syllogisms_gpt2_6th_conclusion.csv


In [ ]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import re

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the dataset without the "Output Label" column
csv_file_path = 'syllogisms_final.csv'
data = pd.read_csv(csv_file_path)

# Preparing the GPT-2 model and tokenizer
model_name = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Defining a function to generate predictions for each row
def generate_prediction(row):
    # Construct a 5-shot prompt with explicit examples
    five_shot_prompt = (
        "Premise 1: All birds lay eggs.\n"
        "Premise 2: Penguins are birds.\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion: Penguins lay eggs.\n\n"
        "Premise 1: All animals that swim are aquatic.\n"
        "Premise 2: All fishes are animals.\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion: All fishes are aquatic.\n\n"
        "Premise 1: All flowers produce pollen.\n"
        "Premise 2: Sunflowers are flowers.\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion: Sunflowers produce pollen.\n\n"
        "Premise 1: All dogs are mammals.\n"
        "Premise 2: Some poodles are dogs.\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion: Some poodles are mammals.\n\n"
        "Premise 1: All encryption algorithms that use asymmetric keys require a public and a private key.\n"
        "Premise 2: All RSAs are encryption algorithms that use asymmetric keys.\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion: All RSAs require a public and a private key.\n\n"
    )

    # Construct the input prompt for the specific syllogism
    prompt = (
        f"{five_shot_prompt}"
        f"Premise 1: {row['Premise 1']}\n"
        f"Premise 2: {row['Premise 2']}\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion:"
    )

    # Tokenizing the input
    input_ids = tokenizer.encode(prompt, return_tensors='pt', truncation=True, max_length=1024).to(device)

    # Generating the output with a limit on the number of tokens
    output_ids = model.generate(input_ids, max_length=400)

    # Decoding the output
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    print(output_text)
    # Finding all conclusion lines
    conclusions = re.findall(r"Conclusion:.*?\.", output_text)

    # Extracting the 6th occurrence
    if len(conclusions) >= 6:
        sixth_conclusion = conclusions[5][11:] # Removing "Conclusion: " (11 characters)
    else:
        sixth_conclusion = "6th conclusion not found."
    # print(sixth_conclusion)
    return sixth_conclusion

# Applying this function to each row and collecting results
data['6th Conclusion'] = data.apply(generate_prediction, axis=1)

# Truncate spaces and compare
def compare_conclusions(row):
    conclusion_1 = row['Conclusion'].strip()
    conclusion_6th = row['6th Conclusion'].strip()
    return conclusion_1 == conclusion_6th

# Calculate the final score
score = data.apply(compare_conclusions, axis=1).sum()

# Display the final score
print("Final score:", score)

# Saving the revised dataset with only the 6th conclusion
output_csv_path = 'syllogisms_gpt2-S_outputs.csv'
data.to_csv(output_csv_path, index=False)

print("Results saved to", output_csv_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for 

Premise 1: All birds lay eggs.
Premise 2: Penguins are birds.
What is the conclusion drawn from the given two premises?
Conclusion: Penguins lay eggs.

Premise 1: All animals that swim are aquatic.
Premise 2: All fishes are animals.
What is the conclusion drawn from the given two premises?
Conclusion: All fishes are aquatic.

Premise 1: All flowers produce pollen.
Premise 2: Sunflowers are flowers.
What is the conclusion drawn from the given two premises?
Conclusion: Sunflowers produce pollen.

Premise 1: All dogs are mammals.
Premise 2: Some poodles are dogs.
What is the conclusion drawn from the given two premises?
Conclusion: Some poodles are mammals.

Premise 1: All encryption algorithms that use asymmetric keys require a public and a private key.
Premise 2: All RSAs are encryption algorithms that use asymmetric keys.
What is the conclusion drawn from the given two premises?
Conclusion: All RSAs require a public and a private key.

Premise 1: All smartphones are devices that can ac

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Premise 1: All birds lay eggs.
Premise 2: Penguins are birds.
What is the conclusion drawn from the given two premises?
Conclusion: Penguins lay eggs.

Premise 1: All animals that swim are aquatic.
Premise 2: All fishes are animals.
What is the conclusion drawn from the given two premises?
Conclusion: All fishes are aquatic.

Premise 1: All flowers produce pollen.
Premise 2: Sunflowers are flowers.
What is the conclusion drawn from the given two premises?
Conclusion: Sunflowers produce pollen.

Premise 1: All dogs are mammals.
Premise 2: Some poodles are dogs.
What is the conclusion drawn from the given two premises?
Conclusion: Some poodles are mammals.

Premise 1: All encryption algorithms that use asymmetric keys require a public and a private key.
Premise 2: All RSAs are encryption algorithms that use asymmetric keys.
What is the conclusion drawn from the given two premises?
Conclusion: All RSAs require a public and a private key.

Premise 1: All video games require electricity to 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Premise 1: All birds lay eggs.
Premise 2: Penguins are birds.
What is the conclusion drawn from the given two premises?
Conclusion: Penguins lay eggs.

Premise 1: All animals that swim are aquatic.
Premise 2: All fishes are animals.
What is the conclusion drawn from the given two premises?
Conclusion: All fishes are aquatic.

Premise 1: All flowers produce pollen.
Premise 2: Sunflowers are flowers.
What is the conclusion drawn from the given two premises?
Conclusion: Sunflowers produce pollen.

Premise 1: All dogs are mammals.
Premise 2: Some poodles are dogs.
What is the conclusion drawn from the given two premises?
Conclusion: Some poodles are mammals.

Premise 1: All encryption algorithms that use asymmetric keys require a public and a private key.
Premise 2: All RSAs are encryption algorithms that use asymmetric keys.
What is the conclusion drawn from the given two premises?
Conclusion: All RSAs require a public and a private key.

Premise 1: Societies should prioritize resources f

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Premise 1: All birds lay eggs.
Premise 2: Penguins are birds.
What is the conclusion drawn from the given two premises?
Conclusion: Penguins lay eggs.

Premise 1: All animals that swim are aquatic.
Premise 2: All fishes are animals.
What is the conclusion drawn from the given two premises?
Conclusion: All fishes are aquatic.

Premise 1: All flowers produce pollen.
Premise 2: Sunflowers are flowers.
What is the conclusion drawn from the given two premises?
Conclusion: Sunflowers produce pollen.

Premise 1: All dogs are mammals.
Premise 2: Some poodles are dogs.
What is the conclusion drawn from the given two premises?
Conclusion: Some poodles are mammals.

Premise 1: All encryption algorithms that use asymmetric keys require a public and a private key.
Premise 2: All RSAs are encryption algorithms that use asymmetric keys.
What is the conclusion drawn from the given two premises?
Conclusion: All RSAs require a public and a private key.

Premise 1: Governments should focus on rapid econo

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Premise 1: All birds lay eggs.
Premise 2: Penguins are birds.
What is the conclusion drawn from the given two premises?
Conclusion: Penguins lay eggs.

Premise 1: All animals that swim are aquatic.
Premise 2: All fishes are animals.
What is the conclusion drawn from the given two premises?
Conclusion: All fishes are aquatic.

Premise 1: All flowers produce pollen.
Premise 2: Sunflowers are flowers.
What is the conclusion drawn from the given two premises?
Conclusion: Sunflowers produce pollen.

Premise 1: All dogs are mammals.
Premise 2: Some poodles are dogs.
What is the conclusion drawn from the given two premises?
Conclusion: Some poodles are mammals.

Premise 1: All encryption algorithms that use asymmetric keys require a public and a private key.
Premise 2: All RSAs are encryption algorithms that use asymmetric keys.
What is the conclusion drawn from the given two premises?
Conclusion: All RSAs require a public and a private key.

Premise 1: Only individuals who conform to traditi

In [ ]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import re

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the dataset without the "Output Label" column
csv_file_path = 'syllogisms_final.csv'
data = pd.read_csv(csv_file_path)

# Preparing the GPT-2 model and tokenizer
model_name = 'gpt2-large'
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Defining a function to generate predictions for each row
def generate_prediction(row):
    # Construct a 5-shot prompt with explicit examples
    five_shot_prompt = (
        "Premise 1: All birds lay eggs.\n"
        "Premise 2: Penguins are birds.\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion: Penguins lay eggs.\n\n"
        "Premise 1: All animals that swim are aquatic.\n"
        "Premise 2: All fishes are animals.\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion: All fishes are aquatic.\n\n"
        "Premise 1: All flowers produce pollen.\n"
        "Premise 2: Sunflowers are flowers.\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion: Sunflowers produce pollen.\n\n"
        "Premise 1: All dogs are mammals.\n"
        "Premise 2: Some poodles are dogs.\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion: Some poodles are mammals.\n\n"
        "Premise 1: All encryption algorithms that use asymmetric keys require a public and a private key.\n"
        "Premise 2: All RSAs are encryption algorithms that use asymmetric keys.\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion: All RSAs require a public and a private key.\n\n"
    )

    # Construct the input prompt for the specific syllogism
    prompt = (
        f"{five_shot_prompt}"
        f"Premise 1: {row['Premise 1']}\n"
        f"Premise 2: {row['Premise 2']}\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion:"
    )

    # Tokenizing the input
    input_ids = tokenizer.encode(prompt, return_tensors='pt', truncation=True, max_length=1024).to(device)

    # Generating the output with a limit on the number of tokens
    output_ids = model.generate(input_ids, max_length=400)

    # Decoding the output
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    # print(output_text)
    # Finding all conclusion lines
    conclusions = re.findall(r"Conclusion:.*?\.", output_text)

    # Extracting the 6th occurrence
    if len(conclusions) >= 6:
        sixth_conclusion = conclusions[5][11:] # Removing "Conclusion: " (11 characters)
    else:
        sixth_conclusion = "6th conclusion not found."
    print(sixth_conclusion)
    return sixth_conclusion

# Applying this function to each row and collecting results
data['6th Conclusion'] = data.apply(generate_prediction, axis=1)

# Truncate spaces and compare
def compare_conclusions(row):
    conclusion_1 = row['Conclusion'].strip()
    conclusion_6th = row['6th Conclusion'].strip()
    return conclusion_1 == conclusion_6th

# Calculate the final score
score = data.apply(compare_conclusions, axis=1).sum()

# Display the final score
print("Final score:", score)

# Saving the revised dataset with only the 6th conclusion
output_csv_path = 'syllogisms_gpt2_outputs.csv'
data.to_csv(output_csv_path, index=False)

print("Results saved to", output_csv_path)

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

 Societies should prioritize resources for the preservation of their majority culture.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 Deregulating labor laws allows for rapid economic growth.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 Only individuals who conform to traditional societal norms should have access to positions of power.
 Some children with neurodiverse conditions deviate from societal expectations.
Final score: 19
Results saved to syllogisms_gpt2_outputs.csv


In [ ]:
import pandas as pd
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
import re

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the dataset without the "Output Label" column
csv_file_path = 'syllogisms_final.csv'
data = pd.read_csv(csv_file_path)

# Preparing the T5 model and tokenizer
model_name = 't5-large'
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
tokenizer = T5Tokenizer.from_pretrained(model_name)

# Function to generate predictions for each row
def generate_prediction(row):
    # Construct a 5-shot prompt with explicit examples
    five_shot_prompt = (
        "Premise 1: All birds lay eggs.\n"
        "Premise 2: Penguins are birds.\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion: Penguins lay eggs.\n\n"
        "Premise 1: All animals that swim are aquatic.\n"
        "Premise 2: Fish are animals.\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion: Fish are aquatic.\n\n"
        "Premise 1: All flowers produce pollen.\n"
        "Premise 2: Sunflowers are flowers.\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion: Sunflowers produce pollen.\n\n"
        "Premise 1: All dogs are mammals.\n"
        "Premise 2: Some poodles are dogs.\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion: Some poodles are mammals.\n\n"
        "Premise 1: All encryption algorithms that use asymmetric keys require a public and a private key.\n"
        "Premise 2: RSA is an encryption algorithm that uses asymmetric keys.\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion: RSA requires a public and a private key.\n\n"
    )

    # Construct the input prompt for the specific syllogism
    prompt = (
        f"{five_shot_prompt}"
        f"Premise 1: {row['Premise 1']}\n"
        f"Premise 2: {row['Premise 2']}\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion:"
    )

    # Tokenizing the input
    input_ids = tokenizer.encode(prompt, return_tensors='pt', truncation=True, max_length=1024).to(device)

    # Generating the output with a limit on the number of tokens
    output_ids = model.generate(input_ids, max_length=400)

    # Decoding the output
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    print(output_text)
    # Finding all conclusion lines
    conclusions = re.findall(r"Conclusion:.*?\.", output_text)

    # Extracting the 6th occurrence
    if len(conclusions) >= 6:
        sixth_conclusion = conclusions[5][11:]  # Removing "Conclusion: " (11 characters)
    else:
        sixth_conclusion = "6th conclusion not found."

    return sixth_conclusion

# Applying this function to each row and collecting results
data['6th Conclusion'] = data.apply(generate_prediction, axis=1)

# Function to truncate spaces and compare conclusions
def compare_conclusions(row):
    conclusion_1 = row['Conclusion'].strip()
    conclusion_6th = row['6th Conclusion'].strip()
    return conclusion_1 == conclusion_6th

# Calculate the final score
score = data.apply(compare_conclusions, axis=1).sum()

# Display the final score
print("Final score:", score)

# Saving the revised dataset with only the 6th conclusion
output_csv_path = 'syllogisms_t5-L_outputs.csv'
data.to_csv(output_csv_path, index=False)

print("Results saved to", output_csv_path)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


True
Not_duplicate
RSA requires a public and a private key.
True
Premise 1: All animals that swim are aquatic. Premise 2: Some poodles are dogs. What is the conclusion drawn from the given two premises? Conclusion: 5G enhances video streaming quality.
False
Not_duplicate
False
Not_duplicate
RSA requires a public and a private key.
Premise 1: All animals that swim are aquatic. Premise 2: All flowers produce pollen. What is the conclusion drawn from the given two premises? Conclusion: All flowers produce pollen.
Premise 1: All animals that swim are aquatic. Premise 2: All flowers produce pollen. Conclusion: All flowers produce pollen.
Premise 1: All animals that swim are aquatic. Premise 2: All flowers produce pollen.
False
Not_duplicate
Premise 1: All animals that swim are aquatic. Premise 2: Sunflowers are flowers. What is the conclusion drawn from the given two premises? Conclusion: Sunflowers produce pollen.
False
RSA requires a public and a private key.
Premise 1: All animals that s

In [ ]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import re

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the dataset without the "Output Label" column
csv_file_path = 'syllogisms_final.csv'
data = pd.read_csv(csv_file_path)

# Preparing the GPT-2 model and tokenizer
model_name = 'gpt2-xl'
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Defining a function to generate predictions for each row
def generate_prediction(row):
    # Construct a 5-shot prompt with explicit examples
    five_shot_prompt = (
        "Premise 1: All birds lay eggs.\n"
        "Premise 2: Penguins are birds.\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion: Penguins lay eggs.\n\n"
        "Premise 1: All animals that swim are aquatic.\n"
        "Premise 2: All fishes are animals.\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion: All fishes are aquatic.\n\n"
        "Premise 1: All flowers produce pollen.\n"
        "Premise 2: Sunflowers are flowers.\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion: Sunflowers produce pollen.\n\n"
        "Premise 1: All dogs are mammals.\n"
        "Premise 2: Some poodles are dogs.\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion: Some poodles are mammals.\n\n"
        "Premise 1: All encryption algorithms that use asymmetric keys require a public and a private key.\n"
        "Premise 2: All RSAs are encryption algorithms that use asymmetric keys.\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion: All RSAs require a public and a private key.\n\n"
    )

    # Construct the input prompt for the specific syllogism
    prompt = (
        f"{five_shot_prompt}"
        f"Premise 1: {row['Premise 1']}\n"
        f"Premise 2: {row['Premise 2']}\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion:"
    )

    # Tokenizing the input
    input_ids = tokenizer.encode(prompt, return_tensors='pt', truncation=True, max_length=1024).to(device)

    # Generating the output with a limit on the number of tokens
    output_ids = model.generate(input_ids, max_length=400)

    # Decoding the output
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    print(output_text)
    # Finding all conclusion lines
    conclusions = re.findall(r"Conclusion:.*?\.", output_text)

    # Extracting the 6th occurrence
    if len(conclusions) >= 6:
        sixth_conclusion = conclusions[5][11:] # Removing "Conclusion: " (11 characters)
    else:
        sixth_conclusion = "6th conclusion not found."
    print(sixth_conclusion)
    return sixth_conclusion

# Applying this function to each row and collecting results
data['6th Conclusion'] = data.apply(generate_prediction, axis=1)

# Truncate spaces and compare
def compare_conclusions(row):
    conclusion_1 = row['Conclusion'].strip()
    conclusion_6th = row['6th Conclusion'].strip()
    return conclusion_1 == conclusion_6th

# Calculate the final score
score = data.apply(compare_conclusions, axis=1).sum()

# Display the final score
print("Final score:", score)

# Saving the revised dataset with only the 6th conclusion
output_csv_path = 'syllogisms_gpt2-XL_outputs.csv'
data.to_csv(output_csv_path, index=False)

print("Results saved to", output_csv_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for 

Premise 1: All birds lay eggs.
Premise 2: Penguins are birds.
What is the conclusion drawn from the given two premises?
Conclusion: Penguins lay eggs.

Premise 1: All animals that swim are aquatic.
Premise 2: All fishes are animals.
What is the conclusion drawn from the given two premises?
Conclusion: All fishes are aquatic.

Premise 1: All flowers produce pollen.
Premise 2: Sunflowers are flowers.
What is the conclusion drawn from the given two premises?
Conclusion: Sunflowers produce pollen.

Premise 1: All dogs are mammals.
Premise 2: Some poodles are dogs.
What is the conclusion drawn from the given two premises?
Conclusion: Some poodles are mammals.

Premise 1: All encryption algorithms that use asymmetric keys require a public and a private key.
Premise 2: All RSAs are encryption algorithms that use asymmetric keys.
What is the conclusion drawn from the given two premises?
Conclusion: All RSAs require a public and a private key.

Premise 1: All smartphones are devices that can ac

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Premise 1: All birds lay eggs.
Premise 2: Penguins are birds.
What is the conclusion drawn from the given two premises?
Conclusion: Penguins lay eggs.

Premise 1: All animals that swim are aquatic.
Premise 2: All fishes are animals.
What is the conclusion drawn from the given two premises?
Conclusion: All fishes are aquatic.

Premise 1: All flowers produce pollen.
Premise 2: Sunflowers are flowers.
What is the conclusion drawn from the given two premises?
Conclusion: Sunflowers produce pollen.

Premise 1: All dogs are mammals.
Premise 2: Some poodles are dogs.
What is the conclusion drawn from the given two premises?
Conclusion: Some poodles are mammals.

Premise 1: All encryption algorithms that use asymmetric keys require a public and a private key.
Premise 2: All RSAs are encryption algorithms that use asymmetric keys.
What is the conclusion drawn from the given two premises?
Conclusion: All RSAs require a public and a private key.

Premise 1: All video games require electricity to 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Premise 1: All birds lay eggs.
Premise 2: Penguins are birds.
What is the conclusion drawn from the given two premises?
Conclusion: Penguins lay eggs.

Premise 1: All animals that swim are aquatic.
Premise 2: All fishes are animals.
What is the conclusion drawn from the given two premises?
Conclusion: All fishes are aquatic.

Premise 1: All flowers produce pollen.
Premise 2: Sunflowers are flowers.
What is the conclusion drawn from the given two premises?
Conclusion: Sunflowers produce pollen.

Premise 1: All dogs are mammals.
Premise 2: Some poodles are dogs.
What is the conclusion drawn from the given two premises?
Conclusion: Some poodles are mammals.

Premise 1: All encryption algorithms that use asymmetric keys require a public and a private key.
Premise 2: All RSAs are encryption algorithms that use asymmetric keys.
What is the conclusion drawn from the given two premises?
Conclusion: All RSAs require a public and a private key.

Premise 1: Societies should prioritize resources f

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Premise 1: All birds lay eggs.
Premise 2: Penguins are birds.
What is the conclusion drawn from the given two premises?
Conclusion: Penguins lay eggs.

Premise 1: All animals that swim are aquatic.
Premise 2: All fishes are animals.
What is the conclusion drawn from the given two premises?
Conclusion: All fishes are aquatic.

Premise 1: All flowers produce pollen.
Premise 2: Sunflowers are flowers.
What is the conclusion drawn from the given two premises?
Conclusion: Sunflowers produce pollen.

Premise 1: All dogs are mammals.
Premise 2: Some poodles are dogs.
What is the conclusion drawn from the given two premises?
Conclusion: Some poodles are mammals.

Premise 1: All encryption algorithms that use asymmetric keys require a public and a private key.
Premise 2: All RSAs are encryption algorithms that use asymmetric keys.
What is the conclusion drawn from the given two premises?
Conclusion: All RSAs require a public and a private key.

Premise 1: Governments should focus on rapid econo

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Premise 1: All birds lay eggs.
Premise 2: Penguins are birds.
What is the conclusion drawn from the given two premises?
Conclusion: Penguins lay eggs.

Premise 1: All animals that swim are aquatic.
Premise 2: All fishes are animals.
What is the conclusion drawn from the given two premises?
Conclusion: All fishes are aquatic.

Premise 1: All flowers produce pollen.
Premise 2: Sunflowers are flowers.
What is the conclusion drawn from the given two premises?
Conclusion: Sunflowers produce pollen.

Premise 1: All dogs are mammals.
Premise 2: Some poodles are dogs.
What is the conclusion drawn from the given two premises?
Conclusion: Some poodles are mammals.

Premise 1: All encryption algorithms that use asymmetric keys require a public and a private key.
Premise 2: All RSAs are encryption algorithms that use asymmetric keys.
What is the conclusion drawn from the given two premises?
Conclusion: All RSAs require a public and a private key.

Premise 1: Only individuals who conform to traditi

In [ ]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import re

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the dataset
csv_file_path = 'syllogisms_final.csv'
data = pd.read_csv(csv_file_path)

# Preparing the GPT-J model and tokenizer
model_name = 'EleutherAI/gpt-neo-2.7B'
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Function to generate predictions for each row
def generate_prediction(row):
    # Construct a 5-shot prompt with explicit examples
    five_shot_prompt = (
        "Premise 1: All birds lay eggs.\n"
        "Premise 2: Penguins are birds.\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion: Penguins lay eggs.\n\n"
        "Premise 1: All animals that swim are aquatic.\n"
        "Premise 2: Fish are animals.\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion: Fish are aquatic.\n\n"
        "Premise 1: All flowers produce pollen.\n"
        "Premise 2: Sunflowers are flowers.\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion: Sunflowers produce pollen.\n\n"
        "Premise 1: All dogs are mammals.\n"
        "Premise 2: Some poodles are dogs.\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion: Some poodles are mammals.\n\n"
        "Premise 1: All encryption algorithms that use asymmetric keys require a public and a private key.\n"
        "Premise 2: RSA is an encryption algorithm that uses asymmetric keys.\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion: RSA requires a public and a private key.\n\n"
    )

    # Construct the input prompt for the specific syllogism
    prompt = (
        f"{five_shot_prompt}"
        f"Premise 1: {row['Premise 1']}\n"
        f"Premise 2: {row['Premise 2']}\n"
        "What is the conclusion drawn from the given two premises?\n"
        "Conclusion:"
    )

    # Tokenizing the input
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)

    # Generating the output with a limit on the number of tokens
    output_ids = model.generate(input_ids, max_length=400)

    # Decoding the output
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    print(output_text)
    # Finding all conclusion lines
    conclusions = re.findall(r"Conclusion:.*?\.", output_text)

    # Extracting the 6th occurrence
    if len(conclusions) >= 6:
        sixth_conclusion = conclusions[5][11:]  # Removing "Conclusion: " (11 characters)
    else:
        sixth_conclusion = "6th conclusion not found."

    print(sixth_conclusion)
    return sixth_conclusion

# Applying this function to each row and collecting results
data['6th Conclusion'] = data.apply(generate_prediction, axis=1)

# Function to truncate spaces and compare conclusions
def compare_conclusions(row):
    conclusion_1 = row['Conclusion'].strip()
    conclusion_6th = row['6th Conclusion'].strip()
    return conclusion_1 == conclusion_6th

# Calculate the final score
score = data.apply(compare_conclusions, axis=1).sum()

# Display the final score
print("Final score:", score)

# Saving the revised dataset with only the 6th conclusion
output_csv_path = 'syllogisms_gptNeo_outputs.csv'
data.to_csv(output_csv_path, index=False)

print("Results saved to", output_csv_path)